In [1]:
import pandas as pd
import numpy as np
import math

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.initializers import glorot_uniform
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model

from IPython import display
%matplotlib inline

Using TensorFlow backend.


In [2]:
df = pd.read_pickle("./full_data_set.pkl")

In [3]:
df.head()

,Participant,Handedness,Finger,Task,Matrix,Timestamp,BlobCoords,BlobCount,Blobs,BlobImage,BlobImgFlattened
42,20,left,ring,DRAG,"[[0, 1, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 0, 1, 0]...",1510664676885,"[(11, 15, 14, 18)]",1,"[[[2, 5, 14, 9], [1, 27, 113, 73], [4, 60, 217...","[[2, 5, 14, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[2, 5, 14, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
43,20,left,ring,DRAG,"[[0, 1, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 0, 1, 0]...",1510664676958,"[(10, 14, 14, 18)]",1,"[[[0, 4, 7, 5], [4, 9, 48, 44], [4, 40, 215, 1...","[[0, 4, 7, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...","[0, 4, 7, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
44,20,left,ring,DRAG,"[[0, 1, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 0, 1, 0]...",1510664677286,"[(9, 13, 15, 19)]",1,"[[[4, 5, 10, 5], [7, 27, 83, 48], [7, 83, 243,...","[[4, 5, 10, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[4, 5, 10, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
45,20,left,ring,DRAG,"[[0, 1, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 0, 1, 0]...",1510664677988,"[(7, 11, 16, 20)]",1,"[[[4, 5, 11, 9], [4, 18, 62, 42], [2, 64, 232,...","[[4, 5, 11, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[4, 5, 11, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
46,20,left,ring,DRAG,"[[0, 1, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 0, 1, 0]...",1510664677169,"[(4, 8, 18, 22)]",1,"[[[4, 10, 18, 15], [5, 23, 125, 119], [7, 32, ...","[[4, 10, 18, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[4, 10, 18, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## Preparing data for the target model

In [4]:
# Change the column to take as the feature (i.e., Finger, Handedness)
feature_column = "Handedness"

# Use this filter to specify the fingers to taken into account (e.g., thumb only, thumb/others, etc.)
df = df[df.Finger == "thumb"]

## Create training and test sets

In [5]:
split_factor = 0.8
ptcp_train = df.Participant.unique()[:math.floor(len(df.Participant.unique())*split_factor)]
ptcp_test = df.Participant.unique()[math.floor(len(df.Participant.unique())*split_factor):]

In [6]:
df_train = df[df.Participant.isin(ptcp_train)].copy(deep=True)
df_test = df[df.Participant.isin(ptcp_test)].copy(deep=True)

In [7]:
# Training set: Convert target names into numbers for classification
X = ""
y = ""
target_names = df[feature_column].unique()
for l in target_names:
    df_temp = df_train[(df_train[feature_column] == l)]
    X_temp = [df_temp.BlobImgFlattened.iloc[i] for i in range(len(df_temp))]
    y_temp = np.ones(len(X_temp)) * np.where(target_names == l)[0][0]
    
    if (type(X) is str):
        X = X_temp
        y = y_temp
    else:
        X.extend(X_temp)
        y = np.append(y, y_temp)
train_x = np.array(X)
train_y = np.array(y)


# Test set: Convert target names into numbers for classification
X = ""
y = ""
for l in target_names:
    df_temp = df_test[(df_test[feature_column] == l)]
    X_temp = [df_temp.BlobImgFlattened.iloc[i] for i in range(len(df_temp))]
    y_temp = np.ones(len(X_temp)) * np.where(target_names == l)[0][0]
    
    if (type(X) is str):
        X = X_temp
        y = y_temp
    else:
        X.extend(X_temp)
        y = np.append(y, y_temp)

test_x = np.array(X)
test_y = np.array(y)

## Train and test with Keras

In [8]:
batch_size = 50
num_classes = len(df_train[feature_column].unique())
epochs = 10000 # run until it gets stopped...

# input image dimensions
img_rows, img_cols = 27, 15

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = (train_x, train_y), (test_x, test_y)

In [9]:
# GPU Config
config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True, device_count = {'GPU' : 4})
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction=0.2
config.gpu_options.allocator_type = 'BFC'

In [10]:
with tf.device('/gpu:1'):
    session = tf.Session(config=config)
    K.set_session(session)

    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    # remove all negative values (noise)
    x_train[x_train < 0] = 0
    x_test[x_test < 0] = 0
    
    # max value is 268 for our data set
    x_train /= 268
    x_test /= 268
    
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model = Sequential()

    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu', input_shape = input_shape))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))


    model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.5))


    model.add(Flatten())
    model.add(Dense(256, activation = "relu", 
                    kernel_initializer=glorot_uniform()
                   ))
    model.add(BatchNormalization(axis=-1))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation = "softmax"))
    
    
    optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
    
    # Change to your TensorBoard folder:
    tensorflowFolder = "/home/huyvietle/tensorboardfiles/FingerSpecific_LeftVsRightThumb_118"
    tfbCallback = keras.callbacks.TensorBoard(log_dir=tensorflowFolder, histogram_freq=0,  
              write_graph=True, write_images=True)

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test),
              callbacks=[tfbCallback, learning_rate_reduction])
    score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (74981, 27, 15, 1)
74981 train samples
20590 test samples
Train on 74981 samples, validate on 20590 samples
Epoch 1/10000
74981/74981 [==============================] - 33s 441us/step - loss: 0.5372 - acc: 0.7300 - val_loss: 2.0474 - val_acc: 0.6388
Epoch 2/10000
74981/74981 [==============================] - 30s 404us/step - loss: 0.3561 - acc: 0.8362 - val_loss: 0.3583 - val_acc: 0.8330
Epoch 3/10000
74981/74981 [==============================] - 30s 403us/step - loss: 0.2401 - acc: 0.8975 - val_loss: 0.2298 - val_acc: 0.9066
Epoch 4/10000
74981/74981 [==============================] - 30s 402us/step - loss: 0.2324 - acc: 0.9031 - val_loss: 0.2235 - val_acc: 0.9075
Epoch 5/10000
74981/74981 [==============================] - 30s 400us/step - loss: 0.2186 - acc: 0.9076 - val_loss: 0.2281 - val_acc: 0.9030
Epoch 6/10000
74981/74981 [==============================] - 30s 405us/step - loss: 0.2160 - acc: 0.9084 - val_loss: 0.2336 - val_acc: 0.9008
Epoch 7/10000
74981/74981

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model.save("model.h5")